In [8]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig

In [9]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='Extendi')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name arn:aws:iam::195613527809:root to get Role path.


## Hub Model Configuration

In [10]:
hub = {
	'HF_MODEL_ID':'LukeGPT88/patient-doctor-text-classifier',
	'HF_TASK':'text-classification'
}

## Create Hugging Face Model Class

In [11]:
model_name = 'patient-doctor-text-classifier-model'

huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub, # configuration for loading model from Hub
	role=role, # IAM role with permissions to create an endpoint
  name=model_name, 
)

## Deploy model to SageMaker Serverless Inference

In [12]:
endpoint_name = 'patient-doctor-text-classifier-endpoint'

# Specify MemorySizeInMB and MaxConcurrency in the serverless config object
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=3072, max_concurrency=10,
)

# Serverless inference

predictor = huggingface_model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name=endpoint_name
)

# Server Inference

# predictor = huggingface_model.deploy(
# 	initial_instance_count=1, # number of instances
# 	instance_type='ml.m5.large', # ec2 instance type
#   endpoint_name=endpoint_name,
#   serializer=sagemaker.serializers.JSONSerializer(),
# 	deserializer=sagemaker.deserializers.JSONDeserializer(),
# )

-------!

In [44]:
predictor.predict({
	"inputs": "I feel sick. I need to go to the hospital.",
})

[{'label': 'PATIENT', 'score': 0.9921571016311646}]

## Clean Up

In [7]:
predictor.delete_model()
predictor.delete_endpoint()